In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 8)
!date

%load_ext autoreload
%autoreload 2

Mon Mar 15 09:09:06 PDT 2021


In [2]:
from db_queries import get_population, get_location_metadata, get_ids, get_model_results, get_covariate_estimates


ModuleNotFoundError: No module named 'db_queries'

In [4]:
data_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/lsff_data_estimated.csv'
df = pd.read_csv(data_path)

df.loc[df.location_name=="Vietnam",'location_name'] = "Viet Nam"

In [4]:
# covs = get_ids("covariate")

In [5]:
locations = get_location_metadata(location_set_id=1, gbd_round_id=6, decomp_step="step4")

In [6]:
locations = locations.loc[locations.location_name.isin(df.location_name.unique()),['location_name','location_id']]
locs = locations.location_id.tolist()

In [7]:
locations

,location_name,location_id
446,Egypt,141
458,Sudan,522
466,Bangladesh,161
468,India,163
...,...,...
627,Côte d'Ivoire,205
629,Ghana,207
635,Niger,213
636,Nigeria,214


In [8]:
df = df.merge(locations[['location_id','location_name']], on = 'location_name', how = 'left')

In [9]:
me_ids = get_ids("modelable_entity")

In [10]:
me_ids[(me_ids.modelable_entity_id.isin([24374,24381]))]

,modelable_entity_id,modelable_entity_name,modelable_entity_description
9962,24374,Diet low in whole grains (g/day) 25+,Post ST-GPR age 25+ model used in estimation o...
9969,24381,Diet high in sodium (g/day) 25+,Post ST-GPR age 25+ model used in estimation o...


In [11]:
covariates = get_ids("covariate")

In [12]:
covariates[(covariates.covariate_name_short=="sdi")]

,covariate_id,covariate_name,covariate_name_short,covariate_description
336,881,Socio-demographic Index,sdi,A measure of development estimated via princip...


In [13]:
[i for i in covariates.covariate_name_short if "iodize" in i]

['hh_iodized_salt_pc']

In [14]:
covariates[covariates.covariate_name_short=="hh_iodized_salt_pc"]

,covariate_id,covariate_name,covariate_name_short,covariate_description
21,46,Proportion of households using iodized salt (a...,hh_iodized_salt_pc,"Proportion of households using iodized salt, w..."


In [15]:
sdi = get_covariate_estimates(
    gbd_round_id=6,
    covariate_id=881,
    decomp_step='step4'
)

In [16]:
sdi = sdi.loc[(sdi.location_id.isin(locs)) & (sdi.year_id==2019),['location_id','mean_value']]

In [17]:
sdi = sdi.rename(columns = {'mean_value':'sdi'})

In [18]:
hh_salt = get_covariate_estimates(
    gbd_round_id=6,
    covariate_id=46,
    decomp_step='step4'
)

In [19]:
hh_salt = hh_salt.loc[(hh_salt.location_id.isin(locs)) & (hh_salt.year_id==2019),['location_id','mean_value']]

In [20]:
hh_salt = hh_salt.rename(columns = {'mean_value':'hh_salt'})

In [21]:
def sex_weight(df, index_cols = ['location_id']):
    """
    Assumes one year (2019), all age (22)
    sex weight "mean" var
    """
    
    N = 1
    for col in index_cols:
        N = N * df[col].nunique()
        
    assert(len(df)==2 * N), "provided index_cols + sex aren't a unique key for this df"
    
    pop = get_population(age_group_id=22,
                     sex_id = [1,2], 
                     location_id=locs, 
                     year_id=2019, 
                     gbd_round_id=6, 
                     decomp_step='step4')
    
    
    # add population counts
    df = df.merge(pop[['location_id','sex_id','population']], 
                  on = ['location_id','sex_id'], 
                  how = 'left')
    
    # weight mean
    df['sex_weight'] = df.population / df.groupby(['location_id']).transform('sum').population
    df['mean'] = df['mean'] * df.sex_weight
    return df.groupby(['location_id'])[['mean']].sum().reset_index()
    

In [22]:
whole_grains = get_model_results('epi', 24374, gbd_round_id=6, sex_id = [1,2], age_group_id=22, location_id=locs, year_id = 2019, decomp_step = 'step4')
whole_grains = sex_weight(whole_grains)
whole_grains = whole_grains.rename(columns = {'mean':'whole_grains'})

In [23]:
sodium = get_model_results('epi', 24381, gbd_round_id=6, sex_id = [1,2], age_group_id=22, location_id=locs, year_id = 2019, decomp_step = 'step4')
sodium = sex_weight(sodium)
sodium = sodium.rename(columns = {'mean':'sodium'})

In [24]:
df = df.merge(sodium, on = 'location_id', how = 'left')

In [25]:
df = df.merge(whole_grains, on = 'location_id', how = 'left')

In [26]:
df = df.merge(sdi, on = 'location_id', how = 'left')
df = df.merge(hh_salt, on = 'location_id', how = 'left')

In [27]:
df

,location_name,vehicle,nutrient,standard,percent of population eating vehicle,percent of population eating industrially produced vehicle,percent of population eating fortified vehicle,location_id,sodium,whole_grains,sdi,hh_salt
0,Angola,wheat flour,iron,Unknown,NaN,NaN,NaN,168,0.981117,8.870610,0.464635,0.805634
1,Angola,wheat flour,zinc,Unknown,NaN,NaN,NaN,168,0.981117,8.870610,0.464635,0.805634
2,Angola,wheat flour,folic acid,Unknown,NaN,NaN,NaN,168,0.981117,8.870610,0.464635,0.805634
3,Angola,wheat flour,vitamin b12,Unknown,NaN,NaN,NaN,168,0.981117,8.870610,0.464635,0.805634
...,...,...,...,...,...,...,...,...,...,...,...,...
347,Nigeria,salt,folic acid,NaN,NaN,93.000000,0.0,214,0.908433,8.865245,0.460839,0.935323
348,Pakistan,wheat flour,vitamin a,NaN,97.333333,48.666667,0.0,165,1.427600,1.228010,0.475739,0.748086
349,Sudan,maize flour,vitamin a,NaN,NaN,NaN,NaN,522,1.025511,1.733069,0.511221,0.297488
350,United Republic of Tanzania,maize flour,vitamin a,NaN,93.000000,25.233333,2.5,189,1.157473,16.953706,0.419147,0.799530


In [28]:
fao_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/inputs/FAOSTAT_food_supply_aggregated_oil_2018.csv'
fao = pd.read_csv(fao_path)
fao.loc[(fao.location_name=="Viet Nam"),'location_name'] = "Vietnam"

In [29]:
df = df.merge(fao, on = 'location_name', how = 'left')

In [30]:
parents = get_location_metadata(location_set_id=1, gbd_round_id=6, decomp_step="step4")

In [31]:
parents = parents.loc[(parents.location_id.isin(df.location_id)),['location_id','parent_id']]

In [32]:
parents

,location_id,parent_id
446,141,138
458,522,138
466,161,159
468,163,159
...,...,...
627,205,199
629,207,199
635,213,199
636,214,199


In [33]:
df = df.merge(parents, on = 'location_id')

In [34]:
df

,location_name,vehicle,nutrient,standard,percent of population eating vehicle,percent of population eating industrially produced vehicle,percent of population eating fortified vehicle,location_id,sodium,whole_grains,sdi,hh_salt,fao_maize_and_products,fao_rice_and_products,fao_wheat_and_products,fao_oil_all,parent_id
0,Angola,wheat flour,iron,Unknown,NaN,NaN,NaN,168,0.981117,8.870610,0.464635,0.805634,415.0,80.0,256.0,282.0,167
1,Angola,wheat flour,zinc,Unknown,NaN,NaN,NaN,168,0.981117,8.870610,0.464635,0.805634,415.0,80.0,256.0,282.0,167
2,Angola,wheat flour,folic acid,Unknown,NaN,NaN,NaN,168,0.981117,8.870610,0.464635,0.805634,415.0,80.0,256.0,282.0,167
3,Angola,wheat flour,vitamin b12,Unknown,NaN,NaN,NaN,168,0.981117,8.870610,0.464635,0.805634,415.0,80.0,256.0,282.0,167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,Viet Nam,wheat flour,vitamin a,Mandatory,21.93,21.93,0.0,20,2.620204,33.449852,0.615286,0.712305,NaN,NaN,NaN,NaN,9
348,Viet Nam,wheat flour,vitamin b1,Unknown,21.93,21.93,0.0,20,2.620204,33.449852,0.615286,0.712305,NaN,NaN,NaN,NaN,9
349,Viet Nam,oil,vitamin a,Mandatory,54.50,90.00,NaN,20,2.620204,33.449852,0.615286,0.712305,NaN,NaN,NaN,NaN,9
350,Viet Nam,oil,vitamin d,Unknown,54.50,90.00,0.0,20,2.620204,33.449852,0.615286,0.712305,NaN,NaN,NaN,NaN,9


In [35]:
df.to_csv('lsff_estimated_data_plus_covariates.csv', index = False)